In [33]:
import os
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
import re
import datetime

In [39]:
"""
Input:
    $ A fasta file containing your DNA sequence that you want to edit with gRNAs (eg. pXW487, in fasta format)
    $ A text file containing two columns (no header),
        one for SNP names (eg rs21062170),
        the other for the corresponding SNP locations (the number should point to the SNPs in the fasta file)

Output:
    $ A text file containing 5 columns: SNP, gRNA, PAM, PAM_pos, strand
    The code designs gRNAs that target around the SNPs and make the closest cut possible near the SNPs
"""


'\nInput:\n    $ A fasta file containing your DNA sequence that you want to edit with gRNAs (eg. pXW487, in fasta format)\n    $ A text file containing two columns (no header),\n        one for SNP names (eg rs21062170),\n        the other for the corresponding SNP locations (the number should point to the SNPs in the fasta file)\n\nOutput:\n    $ A text file containing 5 columns: SNP, gRNA, PAM, PAM_pos, strand\n    The code designs gRNAs that target around the SNPs and make the closest cut possible near the SNPs\n'

In [40]:
current_datetime = datetime.datetime.now()
date_time_str = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")

SEQ_FILENAME = 'pXW470.fa'
SNP_FILENAME = 'SNP_pXW470.txt'
OUTPUT_FILENAME = SEQ_FILENAME.split(".")[0] + "_gRNA_" + date_time_str + ".txt"
    
desktop = os.path.expanduser("~/Desktop")
seq_file_path = desktop + '/' + SEQ_FILENAME
snp_file_path = desktop + '/' + SNP_FILENAME
output_file_path = desktop + '/' + OUTPUT_FILENAME

In [41]:
# Load the fasta sequence from the file
def load_fasta_sequence(file_path):
    with open(file_path, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            return str(record.seq).upper()

In [42]:
my_sequence = load_fasta_sequence(seq_file_path)

In [43]:
# Load the SNP file
def load_snps(file_path):
    df = pd.read_csv(file_path, header=None, sep="[ \t]+").dropna(axis=1, how='all') 
    if df.shape[1] != 2:
        print("Error: column number is not 2")
    else:
        if ('rs' or 'Rs') in df[0].iloc[0] or int(df.iloc[0,1].replace(",", "")):
            df.columns = ['SNP', 'loc']
        else:
            df.columns = ['loc', 'SNP']
    if isinstance(df["loc"].iloc[0], str):
        df["loc"] = df["loc"].str.replace(",", "").astype(int)
    return df

In [44]:
snp_df = load_snps(snp_file_path)
snp_df

C:\Users\wxh02\AppData\Local\Temp\ipykernel_5784\2774996437.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, header=None, sep="[ \t]+").dropna(axis=1, how='all')


,SNP,loc
0,rs2239017,2430
1,rs758171,7042
2,rs71441679,9473
3,rs35848523,10310
4,rs2238049,18910
5,rs2238050,19489
6,rs2238051,19957
7,rs1860056,24618
8,rs12423277,27101
9,rs1016388,34650


In [45]:
def find_nearest_pam(sequence, snp_pos, neighbor_window=50):
    neighbor_seq = sequence[snp_pos - neighbor_window:snp_pos + neighbor_window]
    distances_to_50 = []

    pam_patterns = [re.compile("(?=GG)"), re.compile("(?=CC)")]
    strand_labels = ['+', '-']

    for pam_pattern, strand_label in zip(pam_patterns, strand_labels):
        for match in pam_pattern.finditer(neighbor_seq):
            if strand_label == "+":
                pam_pos = match.start() - 1
                cut_site_pos = pam_pos - 3
            else:
                pam_pos = match.start()
                cut_site_pos = pam_pos + 6
            distance_to_50 = abs(cut_site_pos - 50)
            distances_to_50.append((distance_to_50, pam_pos, strand_label))

    distances_to_50.sort(key=lambda x: x[0])  # Sort by distance

    closest_distance_to_50, closest_pam_pos, closest_strand = distances_to_50[0]

    if closest_strand == '+':
        closest_gRNA_sequence = neighbor_seq[closest_pam_pos - 20: closest_pam_pos]
        closest_pam_seq = neighbor_seq[closest_pam_pos:closest_pam_pos + 3]
    else:
        closest_gRNA_sequence = str(Seq(neighbor_seq[closest_pam_pos + 3: closest_pam_pos + 23]).reverse_complement())
        closest_pam_seq = str(Seq(neighbor_seq[closest_pam_pos:closest_pam_pos + 3]).reverse_complement())

    pam_global_pos = snp_pos - 50 + closest_pam_pos

    print("Closest NGG or CCN Position:", closest_pam_pos )
    print("Closest Distance to Cut Site (3 nt upstream of PAM) from SNP:", closest_distance_to_50)
    print("Closest gRNA Sequence:", closest_gRNA_sequence)
    print("closest PAM", closest_pam_seq)
    print("Closest Strand:", closest_strand)


    return closest_gRNA_sequence, closest_pam_seq, pam_global_pos, closest_strand


In [46]:
gRNAs = []
pams = []
pam_positions = []
strands = []

for index, row in snp_df.iterrows():
    snp_position = row["loc"]
    snp_name = row["SNP"]
    print("\n" + snp_name)
    gRNA, pam, pam_global_position, strand = find_nearest_pam(my_sequence,snp_position)
    gRNAs.append(gRNA)
    pams.append(pam)
    pam_positions.append(pam_global_position)
    strands.append(strand)

# Save the gRNAs, PAMs, and strands in a text file

with open(output_file_path, "w") as file:
    # Write the headers as the first line in the file
    file.write("SNP\tgRNA\tPAM\tPAM_pos\tstrand\n")

    # Write SNP data
    for snp_name, gRNA, pam, pam_glo_pos, strand in zip(snp_df["SNP"], gRNAs, pams, pam_positions, strands):
        file.write(f"{snp_name}\t{gRNA}\t{pam}\t{pam_glo_pos}\t{strand}\n")

print("Data saved to", output_file_path)


rs2239017
Closest NGG or CCN Position: 52
Closest Distance to Cut Site (3 nt upstream of PAM) from SNP: 1
Closest gRNA Sequence: GTGAGTCATTTTGGGAGGAG
closest PAM AGG
Closest Strand: +

rs758171
Closest NGG or CCN Position: 49
Closest Distance to Cut Site (3 nt upstream of PAM) from SNP: 4
Closest gRNA Sequence: CCATGCTGTGGCCAAGCAGC
closest PAM GGG
Closest Strand: +

rs71441679
Closest NGG or CCN Position: 50
Closest Distance to Cut Site (3 nt upstream of PAM) from SNP: 3
Closest gRNA Sequence: AAGGCTGAGCATCTGCGCAG
closest PAM TGG
Closest Strand: +

rs35848523
Closest NGG or CCN Position: 55
Closest Distance to Cut Site (3 nt upstream of PAM) from SNP: 2
Closest gRNA Sequence: TATTCTGATAGATAACTCTA
closest PAM TGG
Closest Strand: +

rs2238049
Closest NGG or CCN Position: 55
Closest Distance to Cut Site (3 nt upstream of PAM) from SNP: 2
Closest gRNA Sequence: CTAGGGCTTCTCAGGTTTTG
closest PAM AGG
Closest Strand: +

rs2238050
Closest NGG or CCN Position: 18
Closest Distance to Cut Site (3